In [39]:
import numpy as np
import pandas as pd
#pd.options.display.max_columns = None

<b> function to parse_url which is already given</b>

In [40]:
from bs4 import BeautifulSoup
import urllib
from urllib.request import Request, urlopen

def parse_url(url):
    """
    input: a single url
    output: text content of all paragraphs in the input url
    """
    
    hdr = {'User-Agent': 'Mozilla/5.0'}
    req = Request(url,headers=hdr)
    response = urllib.request.urlopen(req)
    page = response.read()
    soup = BeautifulSoup(page,'lxml')
    paragraphs = soup.find_all('p')
    
    content = ''
    
    for p in paragraphs:
        content += p.get_text() + ' '
    
    content = ' '.join(content.split())
    
    return content

<b>saving the path of the urls in the variables</b>

In [41]:
urls_file = '/content/drive/MyDrive/ML Projects/NLP project /johnmiller1999-attachments/urls.txt'
stopwords_file ='/content/drive/MyDrive/ML Projects/NLP project /johnmiller1999-attachments/stop_words.txt'

<b>Creating the document_term_matrix function</b>

In [42]:
from keras.preprocessing.text import Tokenizer
def document_term_matrix(urls_file, stopwords_file):

  #reading the urls.txt by its variable
  df = pd.read_csv(urls_file, header=None)
  df.columns = df.columns.map(str)
  df = df.rename(columns={'0':'Url'})

  df['Content'] = df['Url'].map(parse_url)  #calling the parse_url function to fetch the data

  df['Content'] = df['Content'].astype(str).str.lower() #converting the content to lowercase
  
  df['Content'] = df['Content'].str.replace('[^A-Za-z0-9]+',' ') #removing special characters

  #loading the stop_words
  my_file = open(stopwords_file, "r")
  content = my_file.readlines()
  stop_words = [s.rstrip() for s in content]

  #function to remove the stop_words from the content
  def remove_stopwords(text):
    text = ' '.join(word for word in text.split() if word not in stop_words)
    return text
  
  
  df['Content'] = df['Content'].apply(remove_stopwords) #applying the remove_stopwords function

  token = Tokenizer(num_words=50000) #Tokenizer is used to find the unique words from the Content
  token.fit_on_texts(df['Content'])

  lis1 = list(token.word_index.keys()) #the unique words extracted are saved in a list

  lis1.sort() #sort is done to maintain the lexicographic order

  df2 = pd.DataFrame(columns=lis1)
  df2['Content'] = df['Content']

  cols = list(df2.columns)
  cols = [cols[-1]] + cols[:-1]
  df2 = df2[cols]

  df2['Url'] = df['Url']

  cols = list(df2.columns)
  cols = [cols[-1]] + cols[:-1]
  df2 = df2[cols]

  #the below loop is used to calculate the frequency of each word in the Content and return the count in its respective column of the dataframe
  for j in range(0,4):
    for i in lis1:
      i = str(i)
      df2[i][j] = df2['Content'][j].count(i)
  
    
  df3 = df2.copy('deep')
  df3['divison_value'] = len(lis1)-(df2 == 0).sum(axis=1)  #division_value is the count of non-zero records from each row which will be used to calculate the tf (term-frequency)
  df3.iloc[:,2:1693] = df3.iloc[:,2:1693].div(df3['divison_value'], axis=0) 

  return df2 , df3 #returning the final dataframes df2 and df3
  



In [43]:
document_term_matrix(urls_file, stopwords_file)

(                                                 Url  \
 0         https://en.wikipedia.org/wiki/Data_science   
 1          https://datajobs.com/what-is-data-science   
 2  https://www.innoarchitech.com/blog/what-is-dat...   
 3  https://www.analyticsvidhya.com/blog/2015/09/a...   
 
                                              Content   0   1 10 11 12 13 14  \
 0  data science interdisciplinary field uses scie...  20  30  1  1  1  1  2   
 1  lots data unlock real value data data science ...   0   2  0  0  0  0  0   
 2  subscribe ai alex youtube channel home ai news...   7   2  0  0  0  0  0   
 3  questions people ask commonly big data data sc...   4   5  0  0  0  1  0   
 
   15 16 17 18  19 1962 1974 1985 1990s 1992 1996 1997 1998   2  20 2001 2002  \
 0  3  2  1  1  11    1    1    2     1    1    1    1    1  37  13    1    1   
 1  0  0  0  0   0    0    0    0     0    0    0    0    0   2   0    0    0   
 2  0  0  0  0   0    0    0    0     0    0    0    0    0   6   3 